<a href="https://colab.research.google.com/github/ucfilho/optim_raianars/blob/master/ABCOptmi_from_C_language_jul_15_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
NP=175
limit=100
D=3
lb=-500
ub=500
runtime=4
maxCycle=300
FoodNumber=int(NP)
f=np.zeros(FoodNumber) 
Foods=np.zeros((FoodNumber,D)) 
solution=np.zeros(D)
fitness=np.zeros(FoodNumber)
trial=np.zeros(FoodNumber)
prob=np.zeros(FoodNumber)
GlobalParams=np.zeros(D)
GlobalMins=np.zeros(runtime)


In [0]:
def CalculateFitness(fun):
    result=0
    if(fun>=0):
      result=1/(fun+1)
    else:
      result=1+fabs(fun)
    return result

In [0]:
def function( sol):
  #Schwefel(x):
  x=sol  
  summ=0
  for i in range(len(x)):
    new=x[i]*np.sin((abs(x[i]))**0.5)
    summ=summ+new
    top=(418.9829*len(x)-summ) 
  
  return top;


In [0]:
def MemorizeBestSource():

  global GlobalMin
  global GlobalParams
  global Foods
 

  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    if (f[i]<GlobalMin):
      GlobalMin=f[i]
      for j in range(D):
        GlobalParams[j]=Foods[i,j]


In [0]:
def init(index):
  global Foods
  global trial
  global f
  global fitness
  global lb
  global ub
  
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  solution=np.zeros(D)
  
  for j in range(D):
    r=np.random.random()
    Foods[index,j]=r*(ub-lb)+lb
    solution[j]=Foods[index,j]
    
  f[index]=function(solution)
  fitness[index]=CalculateFitness(f[index])
  trial[index]=0

In [0]:
def initial():

  global f
  global Foods
  global GlobalMin
  global GlobalParams

  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  for i in range(FoodNumber):
    init(i) 
  GlobalMin=f[0]
  for i in range(D):
    GlobalParams[i]=Foods[0,i]


In [0]:
def SendEmployedBees():

  global Foods
  global trial
  
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    r = np.random.random()
    param2change= int(r*D)
    r = np.random.random()
    neighbour=int(r*D)
    if(neighbour >= NP):
      neighbour=NP-1
      
    while(neighbour==i):
      r = np.random.random()
      neighbour=int(r*FoodNumber)
      if(neighbour >= NP):
        neighbour=NP-1
    for j in range(D):
      solution[j]=Foods[i,j]
#/*v_{ij}=x_{ij}+\phi_{ij}*(x_{kj}-x_{ij}) */      
    r = np.random.random()
    A=Foods[i,param2change]
    B=Foods[neighbour,param2change]
    solution[param2change]=Foods[i,param2change]+(A-B)*(r-0.5)*2;
    if (solution[param2change]<lb):
      solution[param2change]=lb
    if (solution[param2change]>ub):
      solution[param2change]=ub
    ObjValSol=function(solution)
    FitnessSol=CalculateFitness(ObjValSol)
    if(FitnessSol>fitness[i]):
      trial[i]=0
      for j in range(D):
        Foods[i,j]=solution[j]
      f[i]=ObjValSol
      fitness[i]=FitnessSol
    else:
      trial[i]=trial[i]+1 

In [0]:
def CalculateProbabilities():

  global fitness
  global prob
  FoodNumber=len(fitness)
  maxfit=fitness[0]
  for i in range(1,FoodNumber):    
    if (fitness[i]>maxfit):
      maxfit=fitness[i]
  for i in range(FoodNumber):
    prob[i]=(0.9*(fitness[i]/maxfit))+0.1


In [0]:
def SendOnlookerBees():
  
  global trial
  global Foods
  
  D=len(Foods[0,:])
  solution=np.zeros(D)
  FoodNumber=len(Foods[:,0])
  i=0
  t=0
  while(t<FoodNumber):
    r = np.random.random()
    if(r<prob[i]): 
      r = np.random.random()
      t=t+1
      param2change=int(r*D)
      r = np.random.random()
      neighbour=int(r*FoodNumber)

      while(neighbour==i):
        r = np.random.random()
        neighbour=int(r*FoodNumber)
      for j in range(D):
        solution[j]=Foods[i,j]

      r = np.random.random()
      A=Foods[i,param2change]
      B=Foods[neighbour,param2change]
      solution[param2change]=A+(A-B)*(r-0.5)*2

      if (solution[param2change]<lb):
              solution[param2change]=lb
      if (solution[param2change]>ub):
              solution[param2change]=ub
      ObjValSol=function(solution)
      FitnessSol=CalculateFitness(ObjValSol)
              
      if(FitnessSol>fitness[i]):
        trial[i]=0
        for j in range(D):
          Foods[i,j]=solution[j]
        f[i]=ObjValSol
        fitness[i]=FitnessSol
      else:
        trial[i]=trial[i]+1
      i=i+1
      if (i==FoodNumber):
        i=0


In [0]:
def SendScoutBees():
  global Foods
  global trial
  
  FoodNumber=len(Foods[:,0])
  maxtrialindex=0
  for i in range(1,FoodNumber):
    if (trial[i]>trial[maxtrialindex]):
      maxtrialindex=i
  if(trial[maxtrialindex]>=limit):
    init(maxtrialindex)


In [12]:
'''/*Main program of the ABC algorithm*/'''
for run in range(runtime):
  initial()
  MemorizeBestSource()
  mean=0
  for iter in range(maxCycle):
    SendEmployedBees()
    CalculateProbabilities()
    SendOnlookerBees()
    MemorizeBestSource()
    SendScoutBees()

  for j in range(D):
    print("GlobalParam[",j+1,"]:", GlobalParams[j])
  print("run",run+1,"GBest=",GlobalMin," \n");
  GlobalMins[run]=GlobalMin
  #mean=mean+GlobalMins[run]
  mean=mean+GlobalMin
mean=mean/runtime;
print("Means of",runtime,"runs:",mean,"\n")


GlobalParam[ 1 ]: 420.968746144863
GlobalParam[ 2 ]: 420.9687461391222
GlobalParam[ 3 ]: 420.96874644679525
run 1 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.968746258035
GlobalParam[ 2 ]: 420.9687463666237
GlobalParam[ 3 ]: 420.9687462338429
run 2 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.96874661042733
GlobalParam[ 2 ]: 420.9687465166319
GlobalParam[ 3 ]: 420.9687461408325
run 3 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.96874611042347
GlobalParam[ 2 ]: 420.96874591026557
GlobalParam[ 3 ]: 420.96874701331524
run 4 GBest= 3.818269851763034e-05  

Means of 4 runs: 9.545674629407586e-06 



In [13]:
print(np.median(GlobalMins))

3.818269851763034e-05


In [14]:
3.818269851763034e-05/4

9.545674629407586e-06